# Elliptic curve cryptography 
## Tutorial/playground (part 1)

In [ ]:
import keccak from 'keccak';
import * as Utils from './src/util';
import * as ed25519 from '@noble/ed25519';

> treat an elliptic curve group of points like you do vectors:
  you can add/subtract points `(G + H, G - H)` 
  and you can do scalar multiplication with it `(x * G or xG for short)`.
 ...and the analogy stops there.

> HOWEVER, we can have a vector/array OF scalars & OF points (part 2).

> difference #1: the scalars. in vector calculus, the scalar is real
  numbers. on the other hand, our scalar is integers modulo a large
  prime number l. in other words, our scalars are only from 0 to (l - 1)
 (the remainders when any integer is divided by l).
  Here's the l:

In [ ]:
console.log(`l: ${ed25519.CURVE.l}`);

### scalar initialize

In [ ]:
const one = new Utils.Scalar(BigInt("1"));
const two = new Utils.Scalar(BigInt("2"));
const one_hex = await one.get_hex_value();
const two_hex = await two.get_hex_value();

#### Addition

In [ ]:
const result = async (): Promise<void> => {
    const sum = await (await one.add(two_hex)).get_hex_value();
    console.log(`${one_hex} + ${two_hex} = ${sum}`)
}
result();

### Diff


In [ ]:
const result = async (): Promise<void> => {
    const diff = await (await one.subtract(two_hex)).get_value();
    console.log(`${one_hex} - ${two_hex} = ${diff}`)
}
result();

> What?! Not -1? Again, our numbers are only from 0 to (l - 1). Hence -1 becomes "the same" with (l - 1).

### Prod

In [ ]:
const result = async (): Promise<void> => {
    const product = await (await one.multiply(two_hex)).get_hex_value();
    console.log(`${one_hex} * ${two_hex} = ${product}`)
}
result();

> we have something like "division", but we do not use slash.
>instead, inversion (analogous to "reciprocal") is performed on the supposed
> divisor, then perform multiplication.


In [ ]:
const divide = async(a: string, b: string): Promise<void> => {
    const inv = ed25519.utils.invert(await two.get_value(), ed25519.CURVE.l);
    const quot = await Utils.l_overflow_check(await one.get_value() * inv);
    console.log(`${await a} / ${await b} = ${quot}`);
}
divide(one_hex, two_hex);

> ...Yeah this doesn't make much sense. 1/2 becomes "the same" with... that quotient.
> To make sense of this, we multiply the "quotient" and 2. The product should be 1
> like x * (1/x) = 1.

In [ ]:
const divide = async (a: string, b: string): Promise<void> => {
    const inv = ed25519.utils.invert(await two.get_value(), ed25519.CURVE.l);
    const quot = await Utils.l_overflow_check(await one.get_value() * inv);
    const prod2 = await Utils.l_overflow_check(await two.get_value() * quot);
    console.log(`${a} * ${b} = ${prod2}`);
}
divide(one_hex, two_hex);

> exponent is also possible. the power should be a natural number only.

In [ ]:
const three = new Utils.Scalar(BigInt("3"));
const three_hex = await three.get_hex_value();
const result = async (): Promise<void> => {
    const power = await (await two.exp(three_hex)).get_hex_value();
    console.log(`${two_hex} ** ${three_hex} = ${power}`);
}
result();

### Random scalar

In [ ]:
const result = async (): Promise<void> => {
    const s = await Utils.rnd_scalar();
    console.log(await s.get_hex_value());
}
result();

> differences #2: the elliptic curve points. these are actually points (x,y)
  but the x and y are integers modulo another large (not necessarily prime)
  number q
  we usually do not initialize points like we initialize scalar. instead, we use
  either one of the two:
  1) get a random point

In [ ]:
const random_point = async (): Promise<ed25519.Point> => {
    let point;
    try {
        point = ed25519.Point.fromHex(await (await Utils.rnd_scalar()).get_hex_value())
    } catch {
        point = await random_point();
    }
    return point;
}

In [ ]:
const log_point = async (): Promise<void> => {
    console.log(await random_point());
}
log_point();

2. using the "base generator" G
* [crytpography stack exchange](https://crypto.stackexchange.com/questions/27392/base-point-in-ed25519)
* [monero stack exchange](https://monero.stackexchange.com/questions/6050/what-is-the-base-point-g-from-the-whitepaper-and-how-is-it-represented-as-a)

In [ ]:
const base_generator = async (): Promise<void> => {
    console.log(`G: ${ed25519.Point.BASE.toHex()}`
    );
    console.log(ed25519.Point.BASE)
}
base_generator();

> now to produce another point from G (or any other point), we can do, as being said earlier, 
  addition/subtraction of points (G + H, G - H) and scalar multiplication xG for scalar x.

In [ ]:
const gplusg = ed25519.Point.BASE.add(ed25519.Point.BASE);
const gminusg = ed25519.Point.BASE.subtract(ed25519.Point.BASE);
console.log(`G + G = ${gplusg.toHex()}`);
console.log(`G - G = ${gminusg.toHex()}`);

#### zero point

In [ ]:
console.log(ed25519.Point.ZERO.toHex())

> "Are G - G and Z the same?"

In [ ]:
const g_prod = () => {
    for (let i = BigInt("0"); i < BigInt("15"); ++i) {
        const another_point = i === BigInt("0") ? ed25519.Point.ZERO : ed25519.Point.BASE.multiply(BigInt(i))
        console.log(`${i} * G = ${another_point.toHex()}`)
    }
}
g_prod();

> Those last points look "random". This IS a big reason why we use elliptic curves in cryptography:
   
> If I give you a random point P, it is assumed to be 
  impossible to find the x such that `P = xG`. The problem of finding x is called "Discrete Logarithm
  Problem" (DLP) and the impossibility assumption is called Discrete Logarithm (DL) assumption."

### exercise: what is (-1)G + G? 

In [ ]:
// code here

### exercise: what is (1/x)*(xG)? 

In [ ]:
// code here

### exercise: is Z == Z + random_point()?

In [ ]:
// code here

### cryptographic hash functions

Here is an example of hashing with keccak256
Note: Be careful with Buffer, it is easy to hash the string instead of bytes

In [ ]:
// incorrect
const hash = async (): Promise<void> => {
    const s = new Utils.Scalar(BigInt("12"));
    console.log(
        `hash: ${keccak('keccak256')
            .update(await s.get_hex_value()).digest('hex')}`)
}
hash();

In [ ]:
// correct
const hash = async (): Promise<void> => {
    const s = new Utils.Scalar(BigInt("12"));
    console.log(
        `hash scalar: ${keccak('keccak256')
            .update(Buffer.from(await s.get_hex_value(), 'hex')).digest('hex')}`)
    console.log(
        `hash point: ${keccak('keccak256')
            .update(Buffer.from(ed25519.Point.BASE.toHex(), 'hex')).digest('hex')}`)
}
hash();

### exercise: the Diffie-Hellman (DH) key exchange
>  implement DH key exchange (just use variables):
   Alice and Bob wants to share a secret scalar only they would know.
   Using the generator G and hashing, how would they do it?
   show that after the key exchange, Alice and Bob has a shared secret.

In [ ]:
// code here

### exercise: Monero cryptocurrency uses Pedersen commitment to hide amounts in the blockchain.
> implement Pedersen commitment: given a scalar x, it must output a pair `(r, rG + xH)` where r is
  a random scalar. for Monero, r should never be in the blockchain, only the rG + xH is.

> then demonstrate the homomorphicity of Pedersen commitment: show that
  `pedersen(x1) + pedersen(x2) = (r1 + r2)G + (x1 + x2)H` where r1 and r2 are the 'r' output of
  `pedersen(x1) and pedersen(x2)`, respectively.

In [ ]:
// code here

### test for homomorphicity
 > commit2 = (r1 + r2) * G + (x1 + x2) * H

commit1 should be equal to commit2

### exercise: implement Elgamal point encryption scheme.

here's the scenario:
  * Alice must send the point Y to Bob securely. Bob generates a random keypair `(x, xG)`.
  * x is the private key, and `P = xG` is the public key to be shared to Alice. Alice encrypts
  * Y using P, and sends the cipher to Bob. Bob then decrypts the cipher using x.

just like in DH key exchange, just use variables.
  * encryption: given a point Y and point P, it must output a pair `(rG, Y + rP)`where r is a random scalar.
  * decryption: given a cipher pair (C1, C2) and a scalar x, output `Y = C2 - x * C1`.

then demonstrate the homomorphicity of Elgamal encryption scheme. using
  * two plaintexts `69000 * H and 420 * H`, encrypt both separately, then pairwise add the two ciphers,
  * then decrypt the "sum" cipher. what is the decrypted plaintext? 

In [ ]:
// code here

> `expected_dec = Scalar(69420) * H`